### Convert ALOS2 Data to GCOV

In [ ]:
import os
import sys
from pathlib import Path
import subprocess
import fnmatch
import zipfile
import h5py
from string import Template


Choose your scratch folder

In [ ]:
scratch_folder = Path('/scratch/alex_eco_test/') ##change to your folder


In [ ]:
aoi = 'southfork'

aoi_dir = scratch_folder/aoi
ALOS2_dir = aoi_dir / 'ALOS2'
DEM_dir = aoi_dir / 'DEM'
GCOV_dir = aoi_dir / 'GCOV'
TMP_dir = aoi_dir / 'TMP'

Path(aoi_dir).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir).mkdir(parents=True, exist_ok=True)
Path(DEM_dir).mkdir(parents=True, exist_ok=True)
Path(GCOV_dir).mkdir(parents=True, exist_ok=True)
Path(TMP_dir).mkdir(parents=True, exist_ok=True)

print(GCOV_dir)

In [ ]:
list_of_ALOS_SLCs = ['https://cumulus.asf.alaska.edu/L1.1/A4/0000461060_001001_ALOS2406330840-211201.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000461057_001001_ALOS2410470840-211229.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000462196_001001_ALOS2412540840-220112.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000462195_001001_ALOS2414610840-220126.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000469570_001001_ALOS2424960840-220406.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000470073_001001_ALOS2427030840-220420.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000470939_001001_ALOS2429100840-220504.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000471229_001001_ALOS2431170840-220518.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000472194_001001_ALOS2433240840-220601.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000472776_001001_ALOS2435310840-220615.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000473049_001001_ALOS2437380840-220629.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000474580_001001_ALOS2439450840-220713.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000474962_001001_ALOS2441520840-220727.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000476734_001001_ALOS2443590840-220810.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000478941_001001_ALOS2445660840-220824.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000480752_001001_ALOS2447730840-220907.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000483019_001001_ALOS2449800840-220921.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000484607_001001_ALOS2451870840-221005.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000486388_001001_ALOS2453940840-221019.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000487638_001001_ALOS2456010840-221102.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000493581_001001_ALOS2460150840-221130.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000495714_001001_ALOS2462220840-221214.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000497486_001001_ALOS2464290840-221228.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000499756_001001_ALOS2466360840-230111.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000501936_001001_ALOS2468430840-230125.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000503165_001001_ALOS2470500840-230208.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000504382_001001_ALOS2472570840-230222.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000505514_001001_ALOS2474640840-230308.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000506843_001001_ALOS2476710840-230322.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000507907_001001_ALOS2478780840-230405.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000510241_001001_ALOS2480850840-230419.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000511819_001001_ALOS2482920840-230503.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000514846_001001_ALOS2484990840-230517.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000516588_001001_ALOS2487060840-230531.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000517838_001001_ALOS2489130840-230614.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000519712_001001_ALOS2491200840-230628.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000521694_001001_ALOS2493270840-230712.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000524097_001001_ALOS2495340840-230726.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000526555_001001_ALOS2497410840-230809.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000527820_001001_ALOS2499480840-230823.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000529197_001001_ALOS2501550840-230906.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000531054_001001_ALOS2503620840-230920.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000532666_001001_ALOS2505690840-231004.zip']

In [ ]:
for file in list_of_ALOS_SLCs:
    print(file)
    filename = ALOS2_dir / file.split('/')[-1]
    # urllib.request.urlretrieve(SLC, filename)
    if os.path.isfile(filename)==False:
        command = f"wget -P {ALOS2_dir} -q {file}"
        output = subprocess.check_output(command, shell=True)
        # https://cumulus.asf.alaska.edu/L1.1/A4/0000470073_001001_ALOS2427030840-220420.zip

In [ ]:
zipfiles = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(files,'*.zip')]
for file in zipfiles:
    print(file)
    if os.path.isdir(ALOS2_dir / file[:-4]):
        ##check full unzipping
        if sum(os.path.getsize(ALOS2_dir / file[:-4] / f) for f in os.listdir(ALOS2_dir / file[:-4]) if os.path.isfile(ALOS2_dir / file[:-4] /f)) < 6509714637:
            with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/file.split('/')[-1][:-4])
            # os.remove(file)
    else:
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(ALOS2_dir/file.split('/')[-1][:-4])


In [ ]:
ALOS2folders = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(dirnames,'*')]
ALOS2folders

In [ ]:
## Command line option

######## ALOS2 SLC to NISAR RSLC
# for f in /scratch/alex_eco_test/ALOS2/southfork/*/; 
# do python /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i $f -o "${f%.*}".h5; 
# done

######## Get NISAR DEM
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p $f -o "${f%.*}".vrt; 
# done

######## NISAR RSLC to NISAR GCOV
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*.yaml; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py $f; 
# done

In [ ]:
for ALOS2folder in ALOS2folders[1:]:
    ALOS2_id = ALOS2folder.split('/')[-1]
    print('')
    print(ALOS2_id)
    
    ##ALOS2 SLC --> NISAR RSLC
    command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {ALOS2_dir/ALOS2_id}.h5"
    print('')
    print(command)    
    if os.path.isfile(f"{ALOS2_dir/ALOS2_id}.h5")==False or os.path.getsize(f"{ALOS2_dir/ALOS2_id}.h5")<6000000000:
        try: os.remove(f"{ALOS2_dir/ALOS2_id}.h5")
        except:''
        output = subprocess.check_output(command, shell=True)

    
    ## Get NISAR DEM
    command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p {ALOS2_dir/ALOS2_id}.h5 -o {DEM_dir/ALOS2_id}.vrt"
    print('')
    print(command)    

    if os.path.isfile(f"{DEM_dir/ALOS2_id}.vrt")==False:
        output = subprocess.check_output(command, shell=True)

    
    ## NISAR RSLC --> NISAR GCOV
    filein = open('/home/jovyan/gcov_template.yaml')
    template = Template(filein.read())
    replacements = {'inputfile':  f"{ALOS2_dir/ALOS2_id}.h5",
                    'outputfile':  f"{GCOV_dir/ALOS2_id}_gcov.h5",
                    'demfile': f"{DEM_dir/ALOS2_id}.vrt",
                    'tmp': f"{TMP_dir}/"
                    }
    makeoutput = template.substitute(replacements)
    file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_id),'w')
    file.write(makeoutput)
    file.close()
    filein.close()

    command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_id}.yaml"
    print('')
    print(command)
    if os.path.isfile(f"{GCOV_dir/ALOS2_id}_gcov.h5")==False:
        output = subprocess.check_output(command, shell=True)


In [ ]:
NISAR_GCOVs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(GCOV_dir)
                for f in fnmatch.filter(files,'*.h5')]
NISAR_GCOVs